# Import Packages

In [ ]:
import sys
import os
import shutil
import librosa
import glob
import tqdm
import json
import random

from librosa import display as ld

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import backend as K

from sklearn.preprocessing import StandardScaler
from sklearn.externals.joblib import load, dump
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from IPython import display as ipd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Mount Google Drive
#### 1. First mount

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/sound-of-failure

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/sound-of-failure


#### 2. Remount

In [ ]:
%cd ../../..
drive.flush_and_unmount()
drive.mount('/gdrive', force_remount=True)
%cd /gdrive/My Drive/sound-of-failure

/
Mounted at /gdrive
/gdrive/My Drive/sound-of-failure


# Import Own Modules

In [ ]:
sys.path += ['src/01_data_processing', '/gdrive/MyDrive/soundofsilence/src/models']

import spectrogram as spec
import train_test_split as splt
from conv_autoencoder import Autoencoder

hello world
/gdrive/My Drive/sound-of-failure


# Global constants

In [ ]:
# Directories
RAW_DATA_DIR = os.path.join('/gdrive', 'MyDrive', 'mimii_baseline', 'dataset')
BASE_DIR = os.path.join('/gdrive', 'MyDrive', 'sound-of-failure')


# Mel spectrograms
N_MELS = 128
N_FFT = 1024
HOP_LENGTH = 512
POWER = 2.0
WINDOW = 'hann'


# Subsampling
DIM = 32
STEP = 8


# Scaler and cost function
SCALER_TYPE = 'StandardScaler'
LOSS = 'mean_squared_error'


# Optimizer
OPTIMIZER = 'adam'
EPOCHS = 1
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.1


# Data
DB = '6dB'
MACHINE_TYPE = 'valve'
MACHINE_ID = 'id_00'

# Execute preprocessing steps
#### 1. Generate unscaled mel spectrograms (~ 5 minutes)

In [ ]:
spec.make_mel_dirs(BASE_DIR, DB, MACHINE_TYPE, MACHINE_ID)

spec.make_mels(RAW_DATA_DIR, BASE_DIR, 
               DB, MACHINE_TYPE, MACHINE_ID, 
               N_MELS, N_FFT, HOP_LENGTH, POWER, WINDOW)

Generate normal spectrograms and save to /gdrive/MyDrive/sound-of-failure/data/mel_spectrograms/6dB/valve/id_00/normal.
Generate abnormal spectrograms and save to /gdrive/MyDrive/sound-of-failure/data/mel_spectrograms/6dB/valve/id_00/abnormal.


#### 2. Separate train and test files

In [ ]:
train_files, train_labels, test_files, test_labels = splt.make_train_test_split(BASE_DIR, 
                                                                           DB, MACHINE_TYPE, MACHINE_ID, 
                                                                           random_seed=1)

#### 3. Fit scaler to train files

In [ ]:
scaler = spec.create_scaler(SCALER_TYPE)
spec.fit_scaler_to_mel_files(scaler, train_files)

#### 4. Generate training data from scaling and subsampling mel spectrograms

In [ ]:
train_data = splt.generate_train_data(train_files, scaler, DIM, STEP)

In [ ]:
train_data.shape

(32264, 32, 128, 1)

# Modelling

In [ ]:
my_model = Autoencoder(input_shape=(32, 128, 1),
                       num_nodes=[32, 64, 128, 256,],
                       num_kernel=[5, 5, 3, 3],
                       num_strides=[(1, 2), (2, 2), (2, 2), (1, 2),],
                       latent_dim=20)

print(my_model.summary())

This is my autoencoder
Model: "Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder_input (InputLayer)   [(None, 32, 128, 1)]      0         
_________________________________________________________________
Conv_enc0 (Conv2D)           (None, 32, 64, 32)        832       
_________________________________________________________________
BatchNorm_enc0 (BatchNormali (None, 32, 64, 32)        128       
_________________________________________________________________
ReLU_enc0 (ReLU)             (None, 32, 64, 32)        0         
_________________________________________________________________
Conv_enc1 (Conv2D)           (None, 16, 32, 64)        51264     
_________________________________________________________________
BatchNorm_enc1 (BatchNormali (None, 16, 32, 64)        256       
_________________________________________________________________
ReLU_enc1 (ReLU)             (None, 

In [38]:
model = my_model.model

model.compile(loss=LOSS, optimizer=OPTIMIZER)

history = model.fit(train_data,
                    train_data,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=VALIDATION_SPLIT,
                    shuffle=True,
                    verbose=1)

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()
plt.close()

# Evaluation: reconstruction loss

In [16]:
mel_file = train_files[0]

mel = np.load(mel_file)

scaled_mel = spec.apply_scaler_to_mel(scaler, mel)

batch = splt.subsample_from_mel(scaled_mel, DIM, STEP)

batch_prediction = model.predict(batch)

In [ ]:
squared_errors = np.square(batch - batch_prediction)
squared_error.shape

# Inspecting files

In [17]:
normal_wav_files = spec.get_normal_wav_files(RAW_DATA_DIR, DB, MACHINE_TYPE, MACHINE_ID)
abnormal_wav_files = spec.get_abnormal_wav_files(RAW_DATA_DIR, DB, MACHINE_TYPE, MACHINE_ID)

normal_mel_files = spec.get_normal_mel_files(BASE_DIR, DB, MACHINE_TYPE, MACHINE_ID)
abnormal_mel_files = spec.get_abnormal_mel_files(BASE_DIR, DB, MACHINE_TYPE, MACHINE_ID)

In [36]:
i = 0

wav_file = normal_wav_files[i]
mel_file = normal_mel_files[i]

mel = np.load(mel_file)
scaled_mel = spec.apply_scaler_to_mel(scaler, mel)

fig, ax = plt.subplots()
img = ld.specshow(mel, 
                  x_axis='time', 
                  y_axis='mel', 
                  fmax=8000, 
                  ax=ax)

fig.colorbar(img, ax=ax)
plt.title(f'{DB}, {MACHINE_TYPE}, {MACHINE_ID}, {mel_file.split("/")[-1]}')
plt.show()
plt.close()

fig, ax = plt.subplots()
img = ld.specshow(scaled_mel, 
                  x_axis='time', 
                  y_axis='mel', 
                  fmax=8000, 
                  ax=ax)

fig.colorbar(img, ax=ax)
plt.title(f'{DB}, {MACHINE_TYPE}, {MACHINE_ID}, {mel_file.split("/")[-1]}, scaled')
plt.show()
plt.close()

ipd.Audio(wav_file)

Output hidden; open in https://colab.research.google.com to view.

In [37]:
i = 0

wav_file = abnormal_wav_files[i]
mel_file = abnormal_mel_files[i]

mel = np.load(mel_file)
scaled_mel = spec.apply_scaler_to_mel(scaler, mel)

fig, ax = plt.subplots()
img = ld.specshow(mel, 
                  x_axis='time', 
                  y_axis='mel', 
                  fmax=8000, 
                  ax=ax)

fig.colorbar(img, ax=ax)
plt.title(f'{DB}, {MACHINE_TYPE}, {MACHINE_ID}, {mel_file.split("/")[-1]}')
plt.show()
plt.close()

fig, ax = plt.subplots()
img = ld.specshow(scaled_mel, 
                  x_axis='time', 
                  y_axis='mel', 
                  fmax=8000, 
                  ax=ax)

fig.colorbar(img, ax=ax)
plt.title(f'{DB}, {MACHINE_TYPE}, {MACHINE_ID}, {mel_file.split("/")[-1]}, scaled')
plt.show()
plt.close()

ipd.Audio(wav_file)

Output hidden; open in https://colab.research.google.com to view.